In [2]:
# load some packages

from pyproj import Proj, transform
import utm
from osgeo import gdal

import glob, os

import simplekml
from simplekml import Kml, Color

import xarray as xr
import pickle

from shapely.geometry import Polygon, LineString, Point

from skimage.feature import peak_local_max

from skimage import filters
from skimage.morphology import disk

from scipy.ndimage.filters import minimum_filter, maximum_filter, sobel
from scipy.signal import medfilt2d, argrelextrema, detrend
import numpy as np

from matplotlib import pyplot as plt
import matplotlib.cm as cm
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['text.usetex'] = True #Let TeX do the typsetting
plt.rcParams['text.latex.preamble'] = [r'\usepackage{sansmath}', r'\sansmath'] #Force sans-serif math mode (for axes labels)
plt.rcParams['font.family'] = 'sans-serif' # ... for regular text
plt.rcParams['font.sans-serif'] = 'Helvetica' # Choose a nice font here


In [3]:
# changes of convention for direction
def arctan2_to_bearingfromN(A):
    return np.where((A>-np.pi)&(A<=np.pi/2),-A + np.pi/2,-A + np.pi/2 + 2*np.pi)

def bearingfromN_to_windrose(A):
    return np.where((A>=0)&(A<=3*np.pi/2),-A + 3*np.pi/2,-A + 3*np.pi/2 + 2*np.pi)

In [7]:
##################
# USEFUL FOR ERA5 AWS LOADING
##################
# https://github.com/planet-os/notebooks/blob/master/aws/era5-s3-via-boto.ipynb

timestep = 5

#use this to make the landmask, where SST==273.1604 is land
ds = xr.open_dataset('/media/synology2/WANDS/DUNEPOLY/200801_sea_surface_temperature.nc')
SST = np.asarray(ds.sea_surface_temperature[timestep,:,:])

##################
# MAKE BOXES FOR ALL THE GRID CELLS OF ERA5
##################

lon_era = np.asarray(ds.lon)
lat_era = np.asarray(ds.lat)

difflon_era = np.append(np.diff(lon_era)[0],np.diff(lon_era))
difflat_era = np.append(np.diff(lat_era)[0],np.diff(lat_era))

lonBL_era = lon_era-difflon_era/2
latBL_era = lat_era-difflat_era/2

lonBR_era = lon_era+difflon_era/2
latBR_era = lat_era-difflat_era/2

lonTL_era = lon_era-difflon_era/2
latTL_era = lat_era+difflat_era/2

lonTR_era = lon_era+difflon_era/2
latTR_era = lat_era+difflat_era/2

LAT_era,LON_era = np.meshgrid(lat_era,lon_era)

LATBL_era,LONBL_era = np.meshgrid(latBL_era,lonBL_era)
LATBR_era,LONBR_era = np.meshgrid(latBR_era,lonBR_era)
LATTL_era,LONTL_era = np.meshgrid(latTL_era,lonTL_era)
LATTR_era,LONTR_era = np.meshgrid(latTR_era,lonTR_era)

Lon_era = np.ndarray.flatten(LON_era)
Lat_era = np.ndarray.flatten(LAT_era)

LonBL_era = np.ndarray.flatten(LONBL_era)
LatBL_era = np.ndarray.flatten(LATBL_era)
LonBR_era = np.ndarray.flatten(LONBR_era)
LatBR_era = np.ndarray.flatten(LATBR_era)
LonTL_era = np.ndarray.flatten(LONTL_era)
LatTL_era = np.ndarray.flatten(LATTL_era)
LonTR_era = np.ndarray.flatten(LONTR_era)
LatTR_era = np.ndarray.flatten(LATTR_era)

sst   = np.ndarray.flatten(SST.T)
offland = np.argwhere(sst!=273.1604)
onland  = np.argwhere(sst==273.1604)

In [ ]:
# make kmls of polygons of all the era5 land tiles
# used for finding out which tiles are on dune fields
# split into 10 sub-kmls to make size of files managable
subsets = np.arange(0,np.shape(onland)[0]-int(np.shape(onland)[0]/10),int(np.shape(onland)[0]/10))
subsets = np.append(subsets,np.shape(onland)[0]-1)

# loop through subsets
for k in np.arange(0,np.shape(subsets)[0]-1):
    
    # initialize a kml file
    kml = Kml(open=1)
    kml = simplekml.Kml(open=1)

    # initialize types of info to put in the kml
    mboun = kml.newmultigeometry(name="boundary")
    mtile = kml.newmultigeometry(name="tile")
    mlabl = kml.newfolder(name="number")

    # loop through all the tiles on land that will go in this kml 
    for i in onland[subsets[k]:subsets[k+1]]:
        j = i[0]
        # make sure the longitude convention is correct
        # make the polygons, boundaries and their numbers
        if Lon_era[j]>180:
            mlablcoords = [(Lon_era[j]-360, Lat_era[j])]
        else:
            mlablcoords = [(Lon_era[j], Lat_era[j])]
        lo = mlabl.newpoint(name=str(j), coords=mlablcoords)
        lo.style.iconstyle.icon.href = ""
        linecoords = [(LonBR_era[j],LatBR_era[j]),(LonBL_era[j],LatBL_era[j]),(LonTL_era[j],LatTL_era[j]),(LonTR_era[j],LatTR_era[j]),(LonBR_era[j],LatBR_era[j])]
        mboun.newlinestring(coords=linecoords)
        ptile = linecoords
        mtile.newpolygon(outerboundaryis=ptile)
        print('%.0f percent'%(100*j/np.shape(Lon_era)[0]).astype('float'))

    # Style everything
    mboun.style.linestyle.color = Color.black
    mboun.style.linestyle.width = 1
    mtile.style.polystyle.color = Color.changealpha("77", Color.lightblue)
    mtile.style.linestyle.color = Color.changealpha("77", Color.lightblue)

    # save the file
    kml.save('/home/andrew/Desktop/test%d.kml'%k)

In [8]:
# find all the ERA5 tiles on the dune fields
# this was done manually in google earth and information stored in a text file
# empty rows in the text file delineate seperate dune fields

text_file = open('/media/synology2/WANDS/ERA5/era_dune_tiles.txt', 'r')
lines = text_file.readlines()
text_file.close()

ondunes = []
ondune = [0]
for i in lines:
    # some lines of the text file are ranges in tile in N-S columns
    # example 130:135 means tiles 130 through 135 (inclusive)
    if ':' in i:
        start = i.split(':')[0]
        end   = i.split(':')[1]
        end   = end[:-1]
        ondune = np.append(ondune,np.arange(int(start),int(end)+1))
    # empty line implies new dune field below
    elif '\n'==i:
        ondunes.append(ondune[1:])
        ondune = [0]
    else:
        ondune = np.append(ondune,np.asarray(int(i[:-1])))

# after a first try at this algorithm i noticed a lot of the tiles i chose throw spurious results
# this is because:
# - dunes too small
# - no dunes in image, just sand
# - spurious aster data
# - prominent non-aeolian geomorphic features
# this .txt lists all those tiles, here i remove them from the analysis
text_file = open('/media/synology2/WANDS/ERA5/bad_aster_tiles.txt', 'r')
lines = text_file.readlines()
text_file.close()

# loop through bad tiles
for i in lines:
    # get rid of the bad files from the ondunes list which the algorithm iterate over
    for j in np.arange(0,np.shape(ondunes)[0]):
        ondunes[j] = np.delete(ondunes[j],np.squeeze(np.argwhere(ondunes[j]==int(i[:-1]))))

# long list of all ERA5 dune field tiles, not seperated by field, useful for iteration
Ondunes = np.concatenate(ondunes)

In [10]:
# make a .kml of polygons of all the era5 tiles on each dune field
kml = Kml(open=1)
kml = simplekml.Kml(open=1)

# loop through making each dune field have tiles of this color, then they are easily distinguished
col = [Color.lightblue, Color.lightpink, Color.lightyellow, Color.lightgreen]

for k in np.arange(0,np.shape(ondunes)[0]):
    
    mboun = kml.newmultigeometry(name="boundary_df%d"%k)
    mtile = kml.newmultigeometry(name="tile_df%d"%k)
    mlabl = kml.newfolder(name="number_df%d"%k)
    
    for i in ondunes[k]:
        j = i
        if Lon_era[j]>180:
            mlablcoords = [(Lon_era[j]-360, Lat_era[j])]
        else:
            mlablcoords = [(Lon_era[j], Lat_era[j])]
        lo = mlabl.newpoint(name=str(j), coords=mlablcoords)
        lo.style.iconstyle.icon.href = ""
        linecoords = [(LonBR_era[j],LatBR_era[j]),(LonBL_era[j],LatBL_era[j]),(LonTL_era[j],LatTL_era[j]),(LonTR_era[j],LatTR_era[j]),(LonBR_era[j],LatBR_era[j])]
        mboun.newlinestring(coords=linecoords)
        ptile = linecoords
        mtile.newpolygon(outerboundaryis=ptile)
        print('%.0f percent'%(100*j/np.shape(Lon_era)[0]).astype('float'))

    # Style everything
    mboun.style.linestyle.color = Color.black
    mboun.style.linestyle.width = 1
    mtile.style.polystyle.color = Color.changealpha("77", col[k%4])
    mtile.style.linestyle.color = Color.changealpha("77", col[k%4])

kml.save('/home/andrew/Desktop/era_dune_tiles.kml')

4 percent
4 percent
4 percent
4 percent
4 percent
4 percent
4 percent
4 percent
4 percent
4 percent
4 percent
4 percent
4 percent
4 percent
4 percent
99 percent
100 percent
100 percent
100 percent
100 percent
100 percent
100 percent
100 percent
100 percent
100 percent
100 percent
100 percent
100 percent
100 percent
100 percent
100 percent
100 percent
100 percent
0 percent
0 percent
0 percent
0 percent
0 percent
0 percent
0 percent
0 percent
0 percent
0 percent
0 percent
0 percent
0 percent
0 percent
0 percent
0 percent
0 percent
0 percent
0 percent
0 percent
0 percent
0 percent
0 percent
0 percent
0 percent
0 percent
1 percent
1 percent
1 percent
1 percent
1 percent
1 percent
1 percent
1 percent
1 percent
1 percent
1 percent
1 percent
1 percent
1 percent
1 percent
1 percent
1 percent
1 percent
1 percent
1 percent
1 percent
1 percent
1 percent
1 percent
1 percent
1 percent
1 percent
1 percent
1 percent
2 percent
2 percent
2 percent
2 percent
2 percent
2 percent
2 percent
2 percent
2 per

In [11]:
# just for interest
# this is not the true area of dunes, just the tiles
print('percent of dune tiles covering planet:')
print(100*np.shape(Ondunes)[0]/np.shape(Lon_era)[0])
print('percent of dune tiles covering land:')
print(100*np.shape(Ondunes)[0]/np.shape(onland)[0])

percent of dune tiles covering planet:
0.2554931640625
percent of dune tiles covering land:
0.7689425111685869


In [12]:
# make polygons of all the era5 and aster tiles
# used to find which intersect

gridcells_era = []

for i in np.arange(0,np.shape(Lon_era)[0]):
    gridcell = Polygon([(LonBL_era[i],LatBL_era[i]),
                        (LonTL_era[i],LatTL_era[i]),
                        (LonTR_era[i],LatTR_era[i]),
                        (LonBR_era[i],LatBR_era[i])])
    gridcells_era.append(gridcell)

gridcells_aster = []
aster_files = []

os.chdir("/media/synology2/WANDS/ASTER1/")
for file in glob.glob("*_dem.tif"):
    temp = file[8:-8]
    if temp[0]=='N':
        if temp[3]=='E':
            BL = (int(temp[4:]),int(temp[1:3]))
            TL = (int(temp[4:]),int(temp[1:3])+1)
            TR = (int(temp[4:])+1,int(temp[1:3])+1)
            BR = (int(temp[4:])+1,int(temp[1:3]))
        else:
            BL = (-int(temp[4:])+360,int(temp[1:3]))
            TL = (-int(temp[4:])+360,int(temp[1:3])+1)
            TR = (-int(temp[4:])+360+1,int(temp[1:3])+1)
            BR = (-int(temp[4:])+360+1,int(temp[1:3]))
    else:
        if temp[3]=='E':
            BL = (int(temp[4:]),-int(temp[1:3]))
            TL = (int(temp[4:]),-int(temp[1:3])+1)
            TR = (int(temp[4:])+1,-int(temp[1:3])+1)
            BR = (int(temp[4:])+1,-int(temp[1:3]))
        else:
            BL = (-int(temp[4:])+360,-int(temp[1:3]))
            TL = (-int(temp[4:])+360,-int(temp[1:3])+1)
            TR = (-int(temp[4:])+360+1,-int(temp[1:3])+1)
            BR = (-int(temp[4:])+360+1,-int(temp[1:3]))

    aster_files.append(file)
    gridcells_aster.append(Polygon([BL,TL,TR,BR]))

In [ ]:
# make a list of which aster tiles are overlapping with the era5 tiles on dunes
# this can be 1, 2 or 4
Intersect_files = []

for i in np.arange(0,np.shape(Ondunes)[0]):
    print(100*i/np.shape(Ondunes)[0])
    intersect_files = []
    for j in np.arange(0,np.shape(gridcells_aster)[0]):
        if gridcells_era[Ondunes[i]].intersects(gridcells_aster[j]):
            intersect_files.append(aster_files[j])
                    
    Intersect_files.append(intersect_files)
    
for i in np.arange(0,np.shape(Ondunes)[0]):        
    if (min(gridcells_era[Ondunes[i]].exterior.coords.xy[0])<0)&(max(gridcells_era[Ondunes[i]].exterior.coords.xy[0])>0):
        temp = Polygon([(LonBL_era[Ondunes[i]]+360,LatBL_era[Ondunes[i]]),
                        (LonTL_era[Ondunes[i]]+360,LatTL_era[Ondunes[i]]),
                        (LonTR_era[Ondunes[i]]+360,LatTR_era[Ondunes[i]]),
                        (LonBR_era[Ondunes[i]]+360,LatBR_era[Ondunes[i]])])    
        for j in np.arange(0,np.shape(gridcells_aster)[0]):
            if temp.intersects(gridcells_aster[j]):
                Intersect_files[i].append(aster_files[j])

0.0
0.047778308647873864
0.09555661729574773
0.1433349259436216
0.19111323459149546
0.23889154323936931
0.2866698518872432
0.33444816053511706
0.3822264691829909
0.43000477783086477
0.47778308647873863
0.5255613951266125
0.5733397037744864
0.6211180124223602
0.6688963210702341
0.716674629718108
0.7644529383659818
0.8122312470138557
0.8600095556617295
0.9077878643096035
0.9555661729574773
1.0033444816053512
1.051122790253225
1.098901098901099
1.1466794075489728
1.1944577161968466
1.2422360248447204
1.2900143334925944
1.3377926421404682
1.385570950788342
1.433349259436216
1.4811275680840899
1.5289058767319637
1.5766841853798375
1.6244624940277115
1.6722408026755853
1.720019111323459
1.767797419971333
1.815575728619207
1.8633540372670807
1.9111323459149545
1.9589106545628285
2.0066889632107023
2.0544672718585764
2.10224558050645
2.150023889154324
2.197802197802198
2.2455805064500716
2.2933588150979456
2.3411371237458196
2.388915432393693
2.436693741041567
2.484472049689441
2.5322503583373

In [ ]:
#number of low frequency modes that are removed from the image
#this gets rid of large scale trends in the topography that definitely don't come from dunes
edgewidth = 2

#number of bins the dune height calculation is split into
zhistbins = 25

#vectors to find the long and short axis of dunes
angles = np.linspace(0,np.pi,180)
lonbase = np.cos(angles)
latbase = np.sin(angles)

#global projection, needed to find lengths in meters
p1 = Proj(init='epsg:4326', preserve_units=False)

#for filenames to get ERA5 data
years  = np.arange(2008,2017+1)
months = np.arange(1,12+1)

#for wind rose histograms
resA = 180
resU = 6
ubins = np.linspace(0,10,resU)
Ucr = 6
ucrbins = np.linspace(Ucr,2*Ucr,resU)
mA = (np.linspace(0,2*np.pi,resA+1))

# badindices = []
BADindices = []

#loop over dune ERA tiles
for i in np.arange(0,np.shape(Ondunes)[0]):
# for i in np.arange(911,np.shape(Ondunes)[0]):
# for i in redo:

    try:
        #find the extrema lon lat of the ERA tile in question
        lon_sub_min = min(gridcells_era[Ondunes[i]].exterior.coords.xy[0])
        lon_sub_max = max(gridcells_era[Ondunes[i]].exterior.coords.xy[0])
        lat_sub_min = min(gridcells_era[Ondunes[i]].exterior.coords.xy[1])
        lat_sub_max = max(gridcells_era[Ondunes[i]].exterior.coords.xy[1])

        #convert extrema to ASTER convention
        if lon_sub_min>180:
            lon_sub_min = lon_sub_min-360
        if lon_sub_max>180:
            lon_sub_max = lon_sub_max-360

        #loop through all ASTER tiles that overlap with the current ERA tile
        for j in np.arange(0,np.shape(Intersect_files[i])[0]):

            #open the aster dem tile
            ds = gdal.Open('/media/synology2/WANDS/ASTER1/'+Intersect_files[i][j], gdal.GA_ReadOnly) 

            #get heights
            rb = ds.GetRasterBand(1)

            z = rb.ReadAsArray()
            #convert height array to left to right
            z = np.flip(z,axis=0)
            ds.GetGeoTransform()

            #get aster tile lon lat values
            tilecoordinfo = ds.GetGeoTransform()
            lonp = np.arange(0,np.shape(z)[0])
            #min to max and centered
            lon = tilecoordinfo[0] + lonp*tilecoordinfo[1] + tilecoordinfo[1]/2
            latp = np.arange(0,np.shape(z)[1])
            lat = np.flip(tilecoordinfo[3] + latp*tilecoordinfo[5] + tilecoordinfo[5]/2,axis=0)

            #if there's only one overlapping aster tile, no worries
            if j==0:
                Lon = lon
                Lat = lat
                Z   = z
            #if there's more than one overlapping aster tile, merge them
            else:
                Lon = np.vstack((Lon,lon))
                Lat = np.vstack((Lat,lat))
                Z   = np.dstack((Z,z))

        #if there's 4 overlapping tiles, organise them into an ordered mega-tile
        if np.shape(Lon)[0]==4:
            A0 = np.argwhere(Lat==np.min(Lat))[0][0]
            A1 = np.argwhere(Lat==np.min(Lat))[1][0]
            O0 = np.argwhere(Lon==np.min(Lon))[0][0]
            O1 = np.argwhere(Lon==np.min(Lon))[1][0]
            if O0==A0:
                BL=O0
                TL=O1
                BR=A1
            elif O1==A0:
                BL=O1
                TL=O0
                BR=A1
            elif O1==A1:
                BL=O1
                TL=O0
                BR=A0
            elif O0==A1:
                BL=O0
                TL=O1
                BR=A0
            TR = list(set([0,1,2,3]) - set([BL,TL,BR]))[0]

            Lon = np.hstack((Lon[TL][:],Lon[TR]))
            Lat = np.hstack((Lat[BL][:],Lat[TL]))
            z = np.vstack((np.hstack((Z[:,:,BL],Z[:,:,BR])),np.hstack((Z[:,:,TL],Z[:,:,TR]))))

        #if there's 2 overlapping tiles, organise them into an ordered mega-tile
        elif np.shape(Lon)[0]==2:
            if np.all(Lon[0]==Lon[1]):
                B = np.argwhere(Lat==np.min(Lat))[0][0]
                T = np.argwhere(Lat==np.max(Lat))[0][0]

                Lon = Lon[0]
                Lat = np.hstack((Lat[B][:],Lat[T]))
                z = np.vstack((Z[:,:,B],Z[:,:,T]))

            else:
                L = np.argwhere(Lon==np.min(Lon))[0][0]
                R = np.argwhere(Lon==np.max(Lon))[0][0]

                Lon = np.hstack((Lon[L][:],Lon[R]))
                Lat = Lat[0]
                z = np.hstack((Z[:,:,L],Z[:,:,R]))

        # if there's only 1 tile, no worries
        else:
            z = Z

        #only consider aster data on the era tile
        lon_tile = Lon[(Lon>=lon_sub_min)&(Lon<=lon_sub_max)]
        lat_tile = Lat[(Lat>=lat_sub_min)&(Lat<=lat_sub_max)]

        #make the tile square
        latlondiff = np.shape(lon_tile)[0]-np.shape(lat_tile)[0]

        if latlondiff>0:
            Lon_tile = lon_tile[latlondiff:]
            Lat_tile = lat_tile
        elif latlondiff<0:
            Lon_tile = lon_tile
            Lat_tile = lat_tile[-latlondiff:]
        else:
            Lon_tile = lon_tile
            Lat_tile = lat_tile

        #get the topo data on the square era tile
        z_tile = z[np.argwhere(Lat==np.min(Lat_tile))[0][0]:np.argwhere(Lat==np.max(Lat_tile))[0][0]+1,
                     np.argwhere(Lon==np.min(Lon_tile))[0][0]:np.argwhere(Lon==np.max(Lon_tile))[0][0]+1]

        #get the final square aster grid locations for this era tile
        LON_tile,LAT_tile = np.meshgrid(Lon_tile,Lat_tile)

        #################
        #################
        #################

        #make a mask to take out edgewidth-many low frequency modes
        pad = np.pad(np.ones((np.shape(z_tile)[0]-2*edgewidth,np.shape(z_tile)[0]-2*edgewidth)),(edgewidth,edgewidth),mode='constant',constant_values=0)

        #take the fft
        z_fft   = 1/np.shape(z_tile)[1]/np.shape(z_tile)[0]*np.fft.fft2(z_tile)
        z_fft_r = (z_fft.real**2+z_fft.imag**2)**0.5

        #shift high modes to the middle, erase the lowest modes
        z_fft_s = np.fft.fftshift(z_fft_r*pad)

        #get autocorrelation of the topo data via the inverse of the fft
        z_rft_s = np.fft.fftshift(np.fft.ifft2((z_fft*pad)*np.conj((z_fft*pad)))).real*np.shape(z_tile)[1]*np.shape(z_tile)[0]

        #put the origin in the middle of the autcorrelation
        mid_tile = int(np.shape(LON_tile)[0]/2)
        LON_rft_s = LON_tile-LON_tile[mid_tile,mid_tile]
        LAT_rft_s = LAT_tile-LAT_tile[mid_tile,mid_tile]

        #################

        #get 100 levelsets between 0 correlation and the 2nd largest autocorrelation
        #use the 2nd largest, not largest, to ensure the smallest levelset is 2D
        levelsets = np.linspace(0,np.sort(np.ndarray.flatten(z_rft_s))[-2],100)

        #initialize diagnostics
        arearatio = np.empty(100)
        area = np.empty(100)
        polygons = []
        
        #use a circle mask on the autocorrelation to ensure no bias toward cardinal directions
        #the mask has a diameter 2 pixels less than the image, this ensures levelsets don't have open boundaries
        nicedisk = np.pad(disk((np.shape(z_rft_s)[0]-3)/2),1,mode='constant',constant_values=0)

        #loop through the 100 levelset values
        for j in np.arange(0,np.shape(levelsets)[0]):

            #get all the levelsets of the autocorrelation masked by a circle to ensure no bias toward cardinal directions
            cs = plt.contour(LON_rft_s,LAT_rft_s,z_rft_s*nicedisk,[levelsets[j]],colors='k')

            #find the levelset that contains the origin, i.e. the most correlated shape
            for k in range(len(cs.collections[0].get_paths())):
                if Polygon(cs.collections[0].get_paths()[k].vertices).contains(Point(0,0)):
                    break

            #define it as a shape
            polygon = Polygon(cs.collections[0].get_paths()[k].vertices)

            #find the 0 correlation shape to later see how disordered or defected the dune morphology is
            if j == 0:
                zerotharea = polygon.area

            #find the ratio of the shape's convex hull area to its actual area
            arearatio[j] = polygon.convex_hull.area/polygon.area
            #store the shape area
            area[j] = polygon.area

            #store the current shape
            polygons.append(polygon)

            #clear the contour plot out of memory, important
            plt.clf()

        #find the local minima of the hull to real area ratio, this shows where shapes are most convex
        #these tentatively represent real dune morphologies that may be superimposed on each other
        #use a local minima that has a comparison window of 2 levelsets on either side, not 1 because of noise
        pks = peak_local_max(np.max(arearatio)-arearatio,min_distance=2,indices=True)
        #now ensure we ignore spurious morphologies with the following conditions:
        #ignore all shapes where the ratio is 1 apart from the largest area, this avoid spurious tiny morphologies
        pks = np.delete(pks,np.argwhere(arearatio[pks]<1.01)[:,0][:-1])
        #ignore all shapes that are not very convex, arbitrarily defined as a hull area over 10% of real 
        pks = np.delete(pks,np.argwhere(arearatio[pks]>1.1))

        #there may be no local minima that fit this condition because the area is decaying with larger level sets monotonically, in that case:
        if (np.shape(pks)[0]==0) or (np.all(arearatio[pks]<1.01)):
            #take the largest morphology where the hull area is less than 10% of real
            pks = [np.min(np.argwhere(arearatio<1.1))]

        #there are only 1 or 2 aster-observable morphologies in a tile, here ensure we have the largest and smallest ones
        if np.shape(pks)[0]>2:
            pks = np.array([pks[0],pks[-1]])

        #initialize the morphology info storing arrays
        RI = np.empty(np.shape(pks)[0])
        AI = np.empty(np.shape(pks)[0])
        RJ = np.empty(np.shape(pks)[0])
        AJ = np.empty(np.shape(pks)[0])
        DR = np.empty(np.shape(pks)[0])
        Polygons = []

        #loop through the morpologies
        for j in np.arange(0,np.shape(pks)[0]):    
            intx = []
            inty = []

            # loop through directions from 0 to pi
            for k in np.arange(0,np.shape(angles)[0]):
                # creat a line through the origin in the current direction
                line = LineString([(0,0),(lonbase[k],latbase[k])])

                #if there's only one intersection between the line and shape
                try:
                    #store coordinates where the current line and shape intersects
                    intx.append(line.intersection(polygons[pks[j]].boundary).coords.xy[0][0])
                    inty.append(line.intersection(polygons[pks[j]].boundary).coords.xy[1][0])
                except:
                    #if there's more than one intersection loop through them
                    for l in np.arange(0,np.shape(line.intersection(polygons[pks[j]].boundary))[0]):
                        intx.append(line.intersection(polygons[pks[j]].boundary)[l].coords.xy[0][0])
                        inty.append(line.intersection(polygons[pks[j]].boundary)[l].coords.xy[1][0])

            #store the dune morphology
            Polygons.append(polygons[pks[j]])

            intx  = np.asarray(intx)
            inty  = np.asarray(inty)

            #find where the morphology boundary is closest to the origin
            indi  = np.argmin((intx**2+inty**2)**0.5)
            #store that 'minimal axis', the width - it's times 2 since the distance is from the origin and the shape is symmetric
            RI[j] = 2*(intx[indi]**2+inty[indi]**2)**0.5
            #store the direction of the width
            AI[j] = np.arctan2(inty[indi],intx[indi])

            #do same as above for the 'maximal axis', the length
            indj  = np.argmax((intx**2+inty**2)**0.5)
            RJ[j] = 2*(intx[indj]**2+inty[indj]**2)**0.5
            AJ[j] = np.arctan2(inty[indj],intx[indj])

            #for the current morphology find out how small it is relative to the correlated area, to see how expansive the defected area is
            DR[j] = polygons[pks[j]].area/zerotharea

        #store the coordinates of the morpology lengthscales to convert to meters
        I_lon = np.cos(AI)*RI
        I_lat = np.sin(AI)*RI
        J_lon = np.cos(AJ)*RJ
        J_lat = np.sin(AJ)*RJ

        #################

        #initialize the meter unit lengthscales
        I_meter = []
        J_meter = []
        Z_meter = []

        #initialize the height histogram arrays
        Z_hist = []
        Z_bin_edges = []

        #find the local UTM projection for the era5 tile, to properly convert to meters
        localutm = utm.from_latlon(LAT_tile[mid_tile,mid_tile], LON_tile[mid_tile,mid_tile])[2]
        #create a projection mapping
        p2 = Proj(proj="utm",zone=localutm,datum='WGS84', ellps='WGS84', preserve_units=False)
        #loop through the minimal lengthscales
        for j in np.arange(0,np.shape(I_lon)[0]):
            #convert the lengthscale to meters from lon/lat
            X,Y = np.asarray(transform(p1,p2,LON_tile[mid_tile,mid_tile]+[0,I_lon[j]],LAT_tile[mid_tile,mid_tile]+[0,I_lat[j]]))
            I_meter.append(((X[0]-X[1])**2+(Y[0]-Y[1])**2)**0.5)

            #find the range in the real aster topography convolved by a window the size of the current minimum axis
            ztile_min    = minimum_filter(z_tile,size=(int(RI[j]/np.diff(lon_tile)[0]),int(RI[j]/np.diff(lon_tile)[0])))
            ztile_max    = maximum_filter(z_tile,size=(int(RI[j]/np.diff(lon_tile)[0]),int(RI[j]/np.diff(lon_tile)[0])))
            #if we find the most common range, it will be the best approximation of the height of that morphology, find the distribution,
            hist,bin_edges = np.histogram(ztile_max-ztile_min,bins=zhistbins)
            Z_hist.append(hist)
            Z_bin_edges.append(bin_edges)
            #store the most common height as the morphology height
            Z_meter.append(bin_edges[np.argmax(hist)]+np.diff(bin_edges)[np.argmax(hist)]/2)

        #loop through the maximal lengthscales
        for j in np.arange(0,np.shape(J_lon)[0]):
            X,Y = np.asarray(transform(p1,p2,LON_tile[mid_tile,mid_tile]+[0,J_lon[j]],LAT_tile[mid_tile,mid_tile]+[0,J_lat[j]]))
            J_meter.append(((X[0]-X[1])**2+(Y[0]-Y[1])**2)**0.5)

        #################

        #get the eastward and northward winds for the current era tile
        ulist = []
        vlist = []
        for year in years:
            for month in months:        
                ds = xr.open_dataset('/media/synology2/WANDS/ERA5/%04d%02d_eastward_wind_at_10_metres.nc'%(year,month))
                ulist.append(np.asarray(ds.eastward_wind_at_10_metres[:,np.argwhere(lat_era==Lat_era[Ondunes[i]])[0][0],np.argwhere(lon_era==Lon_era[Ondunes[i]])[0][0]]))
                ds = xr.open_dataset('/media/synology2/WANDS/ERA5/%04d%02d_northward_wind_at_10_metres.nc'%(year,month))
                vlist.append(np.asarray(ds.northward_wind_at_10_metres[:,np.argwhere(lat_era==Lat_era[Ondunes[i]])[0][0],np.argwhere(lon_era==Lon_era[Ondunes[i]])[0][0]]))

        #save the decade timeseries of raw orthogonal speeds
        u = np.concatenate(ulist).ravel()
        v = np.concatenate(vlist).ravel()

        #################

        #get wind speed and direction
        U = (u**2+v**2)**0.5
        A = np.arctan2(v,u)
        #swap from anti-clockwise from east to bearing clockwise from north conventions
        AfromN = arctan2_to_bearingfromN(A)

        #initialize histogram bins
        ha1 = []
        ha2 = []

        #swap from bearing clockwise from north to direction wind is coming from conventions
        AforWR = bearingfromN_to_windrose(AfromN)

        #loop through all wind speed bins
        for j in range(0,np.shape(ubins)[0]): 
            #make a histogram of the directions in that bin
            hista1 = np.histogram(AforWR[U>ubins[j]],bins=resA,range=(0,2*np.pi))
            temp1 = np.append(hista1[0],hista1[0][0])
            ha1.append(temp1)

        #loop through wind speeds larger than a critical value of 6 m/s as a proxy for dune-forming winds
        for j in range(0,np.shape(ucrbins)[0]):    
            hista2 = np.histogram(AforWR[U>ucrbins[j]],bins=resA,range=(0,2*np.pi))
            temp2 = np.append(hista2[0],hista2[0][0])
            ha2.append(temp2)

        #################
        #################
        #################

        #save all the important variables into a pickle file per tile
        with open('/media/synology2/WANDS/ERA5/dune_tiles1/tile_%d.pkl'%Ondunes[i], 'wb') as f:
            pickle.dump((gridcells_era[Ondunes[i]],
                        LON_tile, LAT_tile, z_tile,
                        LON_rft_s,LAT_rft_s,z_rft_s,
                        Polygons,
                        RI,AI,RJ,AJ,DR,
                        I_lon,I_lat,J_lon,J_lat,
                        I_meter,J_meter,Z_meter,
                        Z_hist,Z_bin_edges,
                        u,v,U,A,ha1,ha2)
                        , f)

        print('Percent progress:')
        print(100*i/np.shape(Ondunes)[0])
        
    except:
        BADindices.append(i)
        print('ERROR!!')

Percent progress:
0.0
Percent progress:
0.047778308647873864
Percent progress:
0.09555661729574773
Percent progress:
0.1433349259436216
Percent progress:
0.19111323459149546
Percent progress:
0.23889154323936931
Percent progress:
0.2866698518872432
Percent progress:
0.33444816053511706
Percent progress:
0.3822264691829909
Percent progress:
0.43000477783086477
Percent progress:
0.47778308647873863
Percent progress:
0.5255613951266125
Percent progress:
0.5733397037744864
Percent progress:
0.6211180124223602
Percent progress:
0.6688963210702341
Percent progress:
0.716674629718108
Percent progress:
0.7644529383659818
Percent progress:
0.8122312470138557
Percent progress:
0.8600095556617295
Percent progress:
0.9077878643096035
Percent progress:
0.9555661729574773
Percent progress:
1.0033444816053512
Percent progress:
1.051122790253225
Percent progress:
1.098901098901099
Percent progress:
1.1466794075489728
Percent progress:
1.1944577161968466
Percent progress:
1.2422360248447204
Percent pro

<Figure size 432x288 with 0 Axes>